# Project overview
This notebook presents the core concepts developed during my thesis. The goal is to provide a high-level overview of the project and to explain the main ideas behind the code. The code itself is available in the `src` folder.

## Problem statement
The main goal of this project is to develop a method to solve the `path-planning` problem by taking inspiration on chemotaxis and the way bacteria move in their environment. The problem is defined as follows: given a starting configuration a goal configuration, find a path that connects the two configuration without intersecting any obstacle. The path should be as short as possible and should be found in a reasonable amount of time.

Furthermore, we want our method to have the following properties :
- to sample different paths from a given configuration
- to be able to cut through an obstacle if necessary
- to adapt to dynamic environments

This properties are required for the final application wich is surgical planning. In this context, the surgeon needs to be able to explore different paths from a given configuration with a quality mesure associated to it. Furthermore, the device should be able to make its way through an obstacle, as a bone or a muscle, if necessary. Finally, the method should handle incertainty on the obstacle positions since the surgeon can move obstacles during the surgery.

<img src="figures/cochlea.jpg" width=400 height=200 />
<figcaption>Figure 1 : From left to right, (a) an anatomical view of a cochlea with vestibular chamber and canals, (b) a cochlear implent with the array of electrodes (wire) and the digital signal processor (white box)</figcaption>

Our case study is the cochlear implent represented in Figure 1 (b) in which an array of electrodes is bring inside the cochlea (a) to stimulate the cochlear nerve. Therefore, we consider that our path-planning problem can be reduced to finding a collision-free path for the wire containing the electrodes inside a patient head. Furthermore, we consider that the wire can be approximated by a sequence of segments that apply a constraint on the head and that only the displacement of the head is of interest. Therefore, the model we use is a point robot moving in a 3D environment and containes 6 degree of freedom.

## Chemotaxis
Chemotaxis is the movement, observed in a wide range of organisms, indiced in response to a chemical stimulus. The stimulus can be either attractive or repulsive. We will only focus on the attractive case, in which the chemical stimulus is named chemoattractant. 

It is well known that bacteria use chemotaxis to find food (cf. Figure 2). The idea is that bacteria use the gradient of a chemoattractant to be guided toward the food. The gradient of the chemoattractant is computed by the bacteria and is used to control the direction of the movement. The movement is then a combination of a random walk and a directed movement. The random walk is used to explore the environment and the directed movement is used to move toward the food. The random walk is often  modeled by a Brownian motion and the directed movement is modeled by a drift term. The drift term is proportional to the gradient of the chemoattractant. The model is given by the following stochastic differential equation : 
$$dX_t = \sigma dB_t + \mu dt$$
where $X_t$ is the position of the bacteria at time $t$, $\sigma$ is the diffusion coefficient, $B_t$ is a Brownian motion and $\mu$ is the drift term. 

<img src="figures/E.colis.gif" class="center" width=400 height=200 />
<figcaption>Figure 2 : <i>E.Colis</i> bacteria moving toward a chemoattractant located in the left part of the screen.</figcaption> 

For the drift term not to be dependent of the local variation of concentration, it is shown that it can be described by :
$$\mu = \nabla \log C = \frac{\nabla C}{C}$$
where $C$ is the concentration of the chemoattractant. 

Finaly, we can estimate concentration of the chemoattractant at time $t$ by solving the heat equation : 
$$\partial_t C = D\Delta C$$
where $D$ is the diffusion coefficient of the process.


## Attractive field navigation
Using analogie with the chemotaxis study, the robot can be modeled as the organism and the chemical stimulus is the gradient of a scalar field based on the concentration distribution in the environment. The idea is to use the gradient of a scalar field to guide the robot toward a goal configuration. The scalar field is computed by solving the heat equation with a source term at the goal configuration and absorbing boundary. The source term is a Dirac delta function and serves a source for the chemoatractant. The model is given by the following partial differential equation : 

$$\begin{cases} 
\partial_t C = D\Delta C + C_0\delta_{x_0} \\
C(x, 0) = C_0\delta_{x_0} \space \forall x \in \Omega \\
C(x, t) = 0 \space \forall x \in \partial \Omega 
\end{cases}$$